<h2>导入包</h2>

In [1]:
import os
import re
import logging
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import Dataset
import matplotlib.pyplot as plt
from astral import LocationInfo
from astral.sun import sunrise, sunset, dawn, noon, dusk


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

<h2>读取数据</h2>

<h3>比赛数据</h3>

In [2]:
x_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_实际功率数据.csv", encoding="gbk")
info_train = pd.read_csv("../data/A榜-训练集_分布式光伏发电预测_基本信息.csv", encoding="gbk")
x_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_气象变量数据.csv", encoding="gbk")
y_test = pd.read_csv("../data/submit_example.csv", encoding="utf-8")
info_test = pd.read_csv("../data/A榜-测试集_分布式光伏发电预测_基本信息.csv", encoding="gbk")

In [3]:
x_train = pd.merge(x_train, info_train[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_train["时间"] = pd.to_datetime(x_train["时间"], utc=False).dt.tz_localize('Asia/Shanghai')
x_test = pd.merge(x_test, info_test[["光伏用户编号", "装机容量(kW)", "经度", "纬度"]], how="left", on="光伏用户编号")
x_test["时间"] = pd.to_datetime(x_test["时间"], utc=False).dt.tz_localize('Asia/Shanghai')

In [4]:
y_train = y_train.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_train["level_3"] = y_train["level_3"].apply(lambda x: int(x[1:]))
y_train["时间"] = pd.to_datetime(y_train["时间"], utc=False).dt.tz_localize('Asia/Shanghai')
y_train["时间"] = y_train["时间"] + (y_train["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_train = y_train.drop(columns=["level_3"])

y_test = y_test.set_index(["光伏用户编号", "综合倍率", "时间"]).stack().reset_index().rename(columns={0:"target"})
y_test["level_3"] = y_test["level_3"].apply(lambda x: int(x[1:]))
y_test["时间"] = pd.to_datetime(y_test["时间"], utc=False).dt.tz_localize('Asia/Shanghai')
y_test["时间"] = y_test["时间"] + (y_test["level_3"] - 1) * 15 * pd.Timedelta(1, unit="minutes")
y_test = y_test.drop(columns=["level_3"])

In [5]:
df_train = pd.merge(x_train, y_train, on=["光伏用户编号", "时间"], how="left")
df_test = pd.merge(x_test, y_test, on=["光伏用户编号", "时间"], how="left")
df = pd.concat([df_train, df_test], axis=0)
logging.info(df.columns)

2024-03-28 03:05:30,103 : INFO : Index(['光伏用户编号', '时间', '气压(Pa）', '相对湿度（%）', '云量', '10米风速（10m/s）', '10米风向（°)',
       '温度（K）', '辐照强度（J/m2）', '降水（m）', '100m风速（100m/s）', '100m风向（°)',
       '装机容量(kW)', '经度', '纬度', '综合倍率', 'target'],
      dtype='object')


<h3>外部数据</h3>

In [6]:
outer_files = os.listdir("../data/")
outer_files = [x for x in outer_files if re.match(r"^open-meteo-f\d\.csv$", x)]
outer_dfs = list()
for outer_file in outer_files:
    outer_df = pd.read_csv(os.path.join("../data", outer_file), skiprows=3)
    outer_df["time"] = pd.to_datetime(outer_df["time"], utc=False).dt.tz_localize('Asia/Shanghai')
    outer_df = outer_df.set_index("time", drop=True).asfreq("15T")
    outer_df = outer_df.interpolate(method="time", axis=0)

    outer_df = outer_df.reset_index(drop=False).rename(columns={"time": "时间"})
    outer_df["光伏用户编号"] = re.match(r"^open-meteo-(f\d)\.csv$", outer_file).group(1)    
    outer_dfs.append(outer_df)
outer_data = pd.concat(outer_dfs, axis=0)
for column in outer_data.columns:
    logging.info(column)

2024-03-28 03:06:18,441 : INFO : 时间
2024-03-28 03:06:18,442 : INFO : temperature_2m_archive_best_match (°C)
2024-03-28 03:06:18,443 : INFO : relative_humidity_2m_archive_best_match (%)
2024-03-28 03:06:18,443 : INFO : dew_point_2m_archive_best_match (°C)
2024-03-28 03:06:18,444 : INFO : apparent_temperature_archive_best_match (°C)
2024-03-28 03:06:18,444 : INFO : precipitation_archive_best_match (mm)
2024-03-28 03:06:18,445 : INFO : rain_archive_best_match (mm)
2024-03-28 03:06:18,445 : INFO : snowfall_archive_best_match (cm)
2024-03-28 03:06:18,446 : INFO : snow_depth_archive_best_match (m)
2024-03-28 03:06:18,446 : INFO : weather_code_archive_best_match (wmo code)
2024-03-28 03:06:18,446 : INFO : pressure_msl_archive_best_match (hPa)
2024-03-28 03:06:18,447 : INFO : surface_pressure_archive_best_match (hPa)
2024-03-28 03:06:18,447 : INFO : cloud_cover_archive_best_match (%)
2024-03-28 03:06:18,447 : INFO : cloud_cover_low_archive_best_match (%)
2024-03-28 03:06:18,448 : INFO : cloud_

2024-03-28 03:06:18,488 : INFO : soil_temperature_100_to_255cm_era5_seamless (°C)
2024-03-28 03:06:18,489 : INFO : soil_moisture_0_to_7cm_era5_seamless (m³/m³)
2024-03-28 03:06:18,490 : INFO : soil_moisture_7_to_28cm_era5_seamless (m³/m³)
2024-03-28 03:06:18,490 : INFO : soil_moisture_28_to_100cm_era5_seamless (m³/m³)
2024-03-28 03:06:18,490 : INFO : soil_moisture_100_to_255cm_era5_seamless (m³/m³)
2024-03-28 03:06:18,491 : INFO : is_day_era5_seamless ()
2024-03-28 03:06:18,491 : INFO : sunshine_duration_era5_seamless (s)
2024-03-28 03:06:18,491 : INFO : shortwave_radiation_era5_seamless (W/m²)
2024-03-28 03:06:18,492 : INFO : direct_radiation_era5_seamless (W/m²)
2024-03-28 03:06:18,492 : INFO : diffuse_radiation_era5_seamless (W/m²)
2024-03-28 03:06:18,492 : INFO : direct_normal_irradiance_era5_seamless (W/m²)
2024-03-28 03:06:18,492 : INFO : global_tilted_irradiance_era5_seamless (W/m²)
2024-03-28 03:06:18,493 : INFO : terrestrial_radiation_era5_seamless (W/m²)
2024-03-28 03:06:18,4

<h3>整合数据</h3>

In [7]:
df = pd.merge(df, outer_data[[
    "光伏用户编号", 
    "时间",
    "apparent_temperature_archive_best_match (°C)", 
    "surface_pressure_archive_best_match (hPa)",
    "is_day_archive_best_match ()",
    "relative_humidity_2m_archive_best_match (%)",
    "cloud_cover_archive_best_match (%)",
    "global_tilted_irradiance_instant_archive_best_match (W/m²)",
    "precipitation_archive_best_match (mm)",
    "wind_speed_10m_archive_best_match (km/h)",
    "wind_speed_100m_archive_best_match (km/h)",
    "wind_direction_10m_archive_best_match (°)",
    "wind_direction_100m_archive_best_match (°)",
    "sunshine_duration_archive_best_match (s)",
    "weather_code_archive_best_match (wmo code)"
]], how="left", on=["光伏用户编号", "时间"])

df["温度（K）"] = df["apparent_temperature_archive_best_match (°C)"] + 273.15
df["气压(Pa）"] = df["surface_pressure_archive_best_match (hPa)"] * 100
df["是白天"] = df["is_day_archive_best_match ()"].copy()
df["相对湿度（%）"] = df["relative_humidity_2m_archive_best_match (%)"].copy()
df["云量"] = df["cloud_cover_archive_best_match (%)"] / 100
df["辐照强度（J/m2）"] = df["global_tilted_irradiance_instant_archive_best_match (W/m²)"] * 15 * 60
df["降水（m）"] = df["precipitation_archive_best_match (mm)"] / 1000
df["10米风速（10m/s）"] = df["wind_speed_10m_archive_best_match (km/h)"] / 3600 * 100
df["100m风速（100m/s）"] = df["wind_speed_100m_archive_best_match (km/h)"] / 3600 * 10
df["10米风向（°)"] = df["wind_direction_10m_archive_best_match (°)"].copy()
df["100m风向（°)"] = df["wind_direction_100m_archive_best_match (°)"].copy()

df = df.drop(columns=[
    "apparent_temperature_archive_best_match (°C)", 
    "surface_pressure_archive_best_match (hPa)",
    "is_day_archive_best_match ()",
    "relative_humidity_2m_archive_best_match (%)",
    "cloud_cover_archive_best_match (%)",
    "global_tilted_irradiance_instant_archive_best_match (W/m²)",
    "precipitation_archive_best_match (mm)",
    "wind_speed_10m_archive_best_match (km/h)",
    "wind_speed_100m_archive_best_match (km/h)",
    "wind_direction_10m_archive_best_match (°)",
    "wind_direction_100m_archive_best_match (°)"
])

<h2>特征工程</h2>

<h3>时间特征</h3>

In [8]:
df["年"] = df["时间"].dt.year
df["季节"] = df["时间"].dt.quarter
df["月"] = df["时间"].dt.month
df["日"] = df["时间"].dt.day
df["周"] = df["时间"].dt.week
df["分"] = df["时间"].dt.minute // 15 + df["时间"].dt.hour * 4

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [9]:
# df["分_"] = df["分"].copy()
# df = pd.get_dummies(df, columns=["分_"], prefix_sep="")

<h3>根据日出时间日落时间计算时间段</h3>

In [10]:
def solar_time(current_time, dawn_time, sunrise_time, noon_time, sunset_time, dusk_time) -> int:
    """
    根据太阳判断当前时间段\n
    """
    if dawn_time < current_time < sunrise_time:
        return 1
    elif sunrise_time <= current_time < noon_time:
        return 2
    elif noon_time <= current_time < sunset_time:
        return 3
    elif sunset_time <= current_time < dusk_time:
        return 4
    else:
        return 0

In [11]:
# df["地点"] = df.apply(lambda x: LocationInfo(name=x["光伏用户编号"], region="China", timezone="Asia/Shanghai", latitude=x["纬度"], longitude=x["经度"]), axis=1)
# df["黎明时刻"] = df.apply(lambda x: dawn(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["日出时刻"] = df.apply(lambda x: sunrise(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["正午时刻"] = df.apply(lambda x: noon(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["日落时刻"] = df.apply(lambda x: sunset(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["黄昏时刻"] = df.apply(lambda x: dusk(x["地点"].observer, date=x["时间"], tzinfo=x["地点"].timezone), axis=1).dt.tz_convert("Asia/Shanghai")
# df["时间段"] = df.apply(lambda x: solar_time(x["时间"], x["黎明时刻"], x["日出时刻"], x["正午时刻"], x["日落时刻"], x["黄昏时刻"]), axis=1)
# df = df.drop(columns=["地点", "黎明时刻", "日出时刻", "正午时刻", "日落时刻", "黄昏时刻"])

<h3>光伏用户编号</h3>

In [12]:
df["光伏用户编号_"] = df["光伏用户编号"].copy()
df = pd.get_dummies(df, columns=["光伏用户编号_"], prefix_sep="")

<h3>气象特征</h3>

In [13]:
df['100m风速（100m/s）'] = df['100m风速（100m/s）'] * np.sin(np.pi * df['100m风向（°)'] / 180)
# df['cos_100m风速（100m/s）'] = df['100m风速（100m/s）'] * np.cos(np.pi * df['100m风向（°)'] / 180)
df['10米风速（10m/s）'] = df['10米风速（10m/s）'] * np.sin(np.pi * df['10米风向（°)'] / 180)
# df['cos_10米风速（10m/s）'] = df['10米风速（10m/s）'] * np.cos(np.pi * df['10米风向（°)'] / 180)

In [14]:
df["光照/温度"] = df["辐照强度（J/m2）"] / df["温度（K）"]

<h3>历史值特征</h3>

In [15]:
dfs = []
for site, df_site in df.groupby("光伏用户编号"):
    df_site = df_site.sort_values("时间")
    df_site["辐照强度（J/m2） - 1"] = df_site["辐照强度（J/m2）"].shift(1) - df_site["辐照强度（J/m2）"]
    df_site["辐照强度（J/m2） - 8"] = df_site["辐照强度（J/m2）"].shift(8) - df_site["辐照强度（J/m2）"]
#     df_site["辐照强度（J/m2） - 2"] = df_site["辐照强度（J/m2）"].shift(2) - df_site["辐照强度（J/m2）"]
    dfs.append(df_site)
df = pd.concat(dfs, axis=0)

<h3>处理异常值</h3>

In [16]:
print(df_train["target"].nsmallest(3))
df_train[df_train["target"] < -8]
df.loc[207628:207633, ["光伏用户编号", "时间", "target"]]

207630   -8.8900
37316    -0.0085
39139    -0.0084
Name: target, dtype: float64


,光伏用户编号,时间,target
207628,f6,2022-08-15 20:30:00+08:00,-0.002
207629,f6,2022-08-15 20:45:00+08:00,-0.002
207630,f6,2022-08-15 21:00:00+08:00,-8.890
207631,f6,2022-08-15 21:15:00+08:00,-0.002
207632,f6,2022-08-15 21:30:00+08:00,NaN
207633,f6,2022-08-15 21:45:00+08:00,NaN


In [17]:
df.loc[207630, "target"] = -0.002

<h3>光照与当天最强光照的比值</h3>

In [18]:
df["日期"] = df["时间"].dt.date
day_max_values = df[["光伏用户编号", "日期", "辐照强度（J/m2）"]].groupby(by=["光伏用户编号", "日期"]).max()
day_max_values = day_max_values.rename(columns={x: x + "_max" for x in day_max_values.columns}).reset_index()
df = pd.merge(df, day_max_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])
df["辐照强度（J/m2）_max"] = df["辐照强度（J/m2）"] / df["辐照强度（J/m2）_max"]

<h3>当天的平均光照</h3>

In [19]:
df["日期"] = df["时间"].dt.date
day_mean_values = df[["光伏用户编号", "日期", "辐照强度（J/m2）"]].groupby(by=["光伏用户编号", "日期"]).mean()
day_mean_values = day_mean_values.rename(columns={x: x + "_mean" for x in day_mean_values.columns}).reset_index()
df = pd.merge(df, day_mean_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])

<h3>温度与当天最高温最低度的差值</h3>

In [20]:
df["日期"] = df["时间"].dt.date
day_max_values = df[["光伏用户编号", "日期", "温度（K）"]].groupby(by=["光伏用户编号", "日期"]).max()
day_min_values = df[["光伏用户编号", "日期", "温度（K）"]].groupby(by=["光伏用户编号", "日期"]).min()
day_max_values = day_max_values.rename(columns={x: x + "_max" for x in day_max_values.columns}).reset_index()
day_min_values = day_min_values.rename(columns={x: x + "_min" for x in day_min_values.columns}).reset_index()
df = pd.merge(df, day_max_values, on=["光伏用户编号", "日期"], how="left")
df = pd.merge(df, day_min_values, on=["光伏用户编号", "日期"], how="left").drop(columns=["日期"])
df["温度（K）_max"] = df["温度（K）_max"] - df["温度（K）"]
df["温度（K）_min"] = df["温度（K）"] - df["温度（K）_min"]

<h3>划分测试集</h3>

In [21]:
df_train = df[df["时间"] <= df_train["时间"].max()]
df_test = df[df["时间"] >= df_test["时间"].min()]

<h2>训练模型</h2>

<h3>评测指标</h3>

In [22]:
def score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return 1 / (1 + rmse)

<h3>lightgbm模型</h3>

In [23]:
params_lgb = {
    "num_boost_round": 1000,
    'learning_rate': 0.02,
    'boosting_type': 'gbdt',
    'objective': 'mse',
    'metric': 'mse',
    'num_leaves': 127,
    'verbose': -1,
    'seed': 42,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    "early_stopping_round": 100
}

<h3>交叉验证</h3>

In [24]:
model_lgb = []
kfold = KFold(n_splits=5, random_state=42, shuffle=True)

x = df_train.drop(columns=["光伏用户编号", "时间"]).dropna().astype(np.float32)
y = x.pop("target")
mse = 0
for fold, (train_index, val_index) in enumerate(kfold.split(x, y)):
    logging.info(f'############ fold: {fold} ###########')
    x_train, x_val, y_train, y_val = x.iloc[train_index], x.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
    trainset = Dataset(x_train, y_train)
    valset = Dataset(x_val, y_val)
    model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], categorical_feature=["分"], callbacks=[lgb.log_evaluation(1000)])
    model.save_model("../models/lgb_%d.txt" % fold)
    model_lgb.append(model)
    val_pred = Series(model.predict(x_val, num_iteration=model.best_iteration), index=y_val.index).fillna(0)
    mse += mean_squared_error(y_val.fillna(0), val_pred)
rmse = np.sqrt(mse)
score = 1 / (1 + rmse)
logging.info(f"--------------本地分数 {score}--------------")

2024-03-28 03:06:21,792 : INFO : ############ fold: 0 ###########
C:\Program Files\Python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0129875	valid_1's l2: 0.0223124


2024-03-28 03:06:32,299 : INFO : ############ fold: 1 ###########
C:\Program Files\Python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0131567	valid_1's l2: 0.0213376


2024-03-28 03:06:44,890 : INFO : ############ fold: 2 ###########
C:\Program Files\Python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

2024-03-28 03:06:57,588 : INFO : ############ fold: 3 ###########
C:\Program Files\Python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0131067	valid_1's l2: 0.021375


2024-03-28 03:07:10,723 : INFO : ############ fold: 4 ###########
C:\Program Files\Python37\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['分']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Program Files\Python37\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[1000]	training's l2: 0.0129985	valid_1's l2: 0.0224901


2024-03-28 03:07:23,974 : INFO : --------------本地分数 0.7518502185692929--------------


In [25]:
importance = DataFrame()
importance["特征"] = model.feature_name()
importance["重要性"] = 0
for model in model_lgb:
    importance["重要性"] = importance["重要性"] + model.feature_importance()
importance["重要性"] = importance["重要性"] / kfold.n_splits
importance.sort_values("重要性", ascending=False)[0:50]

,特征,重要性
36,辐照强度（J/m2）_mean,10651.2
0,气压(Pa）,9561.8
20,日,7179.4
5,温度（K）,7089.6
2,云量,6969.4
1,相对湿度（%）,6429.6
38,温度（K）_min,5986.8
33,辐照强度（J/m2）_-_1,5527.4
21,周,5150.0
34,辐照强度（J/m2）_-_8,4979.8


<h2>预测</h2>

In [26]:
x_test = df_test.drop(columns=["光伏用户编号", "时间"]).astype(np.float32)
y_test = x_test.pop("target")
y_pred = np.zeros((df_test.shape[0], ))
for model in model_lgb:
    y_pred += model.predict(x_test, num_iteration=model.best_iteration)
y_pred = y_pred / kfold.n_splits
df_test["target"] = y_pred

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [27]:
df_test = df_test[["光伏用户编号", "综合倍率", "年", "月", "日", "分", "target"]]
df_test["时间"] = df_test["年"].astype(str) + "-" + df_test["月"].astype(str) + "-" + df_test["日"].astype(str) + " 0:00"
df_test["分"] = "p" + (df_test["分"] + 1).astype(str)
df_test = df_test.drop(columns=["年", "月", "日"])

In [28]:
result = pd.pivot(df_test, index=["光伏用户编号", "综合倍率", "时间"], columns="分", values="target").reset_index()
result = result[result["综合倍率"].notnull()]
result["综合倍率"] = result["综合倍率"].astype(int)

In [29]:
result.to_csv("../data/%s.csv" % datetime.now().strftime("%Y%m%d_%H%M%S"), encoding="utf-8", index=False)